<a href="https://colab.research.google.com/github/aniuta-mul/Python_Projects/blob/main/A_B_Testing_Tool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd /content/drive/MyDrive/Mate_homework

Mounted at /content/drive
/content/drive/MyDrive/Mate_homework


In [ ]:
import numpy as np
import pandas as pd

ab_test = pd.read_csv("test.csv")
ab_test.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


Виведення потрібних метрик.

In [ ]:
def metric(event_name):
  if event_name == "new account":
    return "new_accounts / session"
  elif event_name == "begin_checkout":
    return "begin_checkout / session"
  elif event_name == "add_shipping_info":
    return "add_shipping_info / session"
  elif event_name == "add_payment_info":
    return "add_payment_info / session"
  elif event_name == "session":
    return "session"
  else:
    np.nan

ab_test["metric"] = ab_test["event_name"].apply(metric)
ab_test.dropna()

,date,country,device,continent,channel,test,test_group,event_name,value,metric
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1,new_accounts / session
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1,new_accounts / session
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1,new_accounts / session
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1,new_accounts / session
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1,new_accounts / session
...,...,...,...,...,...,...,...,...,...,...
800877,2020-12-10,Vietnam,mobile,Asia,Organic Search,3,2,add_payment_info,2,add_payment_info / session
800899,2020-11-21,Vietnam,mobile,Asia,Direct,2,1,add_shipping_info,1,add_shipping_info / session
800900,2020-11-21,Vietnam,mobile,Asia,Direct,3,2,add_shipping_info,1,add_shipping_info / session
800931,2021-01-17,Vietnam,desktop,Asia,Paid Search,4,1,add_shipping_info,2,add_shipping_info / session


Кількість івентів A/B тестів у розрізі тестів і метрик.

In [ ]:
events_df = pd.pivot_table(ab_test[ab_test["metric"].isin(["add_payment_info / session", "add_shipping_info / session",
                                                        "begin_checkout / session", "new_accounts / session"])],
                        values="value", index=["test", "metric"], columns="test_group", aggfunc="sum").reset_index()
events_df = events_df.rename(columns={1: "events_count_test_a", 2: "events_count_test_b"})
events_df.head()

test_group,test,metric,events_count_test_a,events_count_test_b
0,1,add_payment_info / session,1988,2229
1,1,add_shipping_info / session,3034,3221
2,1,begin_checkout / session,3784,4021
3,1,new_accounts / session,3823,3681
4,2,add_payment_info / session,2344,2409


Кількість сесій А/В тестів у розрізі тестів.

In [ ]:
session_df = pd.pivot_table(ab_test[ab_test["metric"].isin(["session"])], values="value", index=["test", "metric"],
                        columns="test_group", aggfunc="sum").reset_index()
session_df = session_df.rename(columns={1: "sessions_count_test_a", 2: "sessions_count_test_b"})
session_df

test_group,test,metric,sessions_count_test_a,sessions_count_test_b
0,1,session,45362,45193
1,2,session,50637,50244
2,3,session,70047,70439
3,4,session,105079,105141


In [ ]:
test_ab = pd.merge(events_df, session_df, on="test", how="left")
test_ab = test_ab.drop(columns="metric_y").rename(columns={"metric_x": "metric"})
test_ab_new = ["test", "metric", "events_count_test_a", "sessions_count_test_a", "events_count_test_b", "sessions_count_test_b"]
test_ab = test_ab[test_ab_new]
test_ab["conversion"] = test_ab["events_count_test_a"] / test_ab["sessions_count_test_a"]
test_ab["metric_change"] = ((test_ab["events_count_test_b"]/test_ab["sessions_count_test_b"]/test_ab["conversion"] )-1)*100
test_ab.head()


test_group,test,metric,events_count_test_a,sessions_count_test_a,events_count_test_b,sessions_count_test_b,conversion,metric_change
0,1,add_payment_info / session,1988,45362,2229,45193,0.043825,12.542021
1,1,add_shipping_info / session,3034,45362,3221,45193,0.066884,6.560481
2,1,begin_checkout / session,3784,45362,4021,45193,0.083418,6.660587
3,1,new_accounts / session,3823,45362,3681,45193,0.084278,-3.354299
4,2,add_payment_info / session,2344,50637,2409,50244,0.046290,3.576911


Розрахунок p-value для визначення статистичної значущості. Якщо p-value < 0.05, то значення є статистично значущим і у колонці significant виведеться значення TRUE, якщо ж  p-value > 0.05, то значення не є статистично значущим і у колонці significant виведеться значення FALSE.

In [ ]:
from pickle import TRUE
import statsmodels.api as sm

result = []
for index, row in test_ab.iterrows():
  count = [row["events_count_test_a"], row["events_count_test_b"]]
  nobs = [row["sessions_count_test_a"], row["sessions_count_test_b"]]
  z_stat, p_value = sm.stats.proportions_ztest(count, nobs)
  result.append((z_stat, p_value))
z_stat_and_p_value = pd.DataFrame(result).reset_index()

test_ab_ = test_ab.join(z_stat_and_p_value)
test_ab_ = test_ab_.drop("index", axis=1).rename(columns={0: "z_stat", 1: "p_value"})
test_ab_

def significant(p_value):
  if p_value > 0.05:
    return "FALSE"
  else:
    return "TRUE"

test_ab_["significant"] = test_ab_["p_value"].apply(significant)
test_ab_

,test,metric,events_count_test_a,sessions_count_test_a,events_count_test_b,sessions_count_test_b,conversion,metric_change,z_stat,p_value,significant
0,1,add_payment_info / session,1988,45362,2229,45193,0.043825,12.542021,-3.924884,0.000087,TRUE
1,1,add_shipping_info / session,3034,45362,3221,45193,0.066884,6.560481,-2.603571,0.009226,TRUE
2,1,begin_checkout / session,3784,45362,4021,45193,0.083418,6.660587,-2.978783,0.002894,TRUE
3,1,new_accounts / session,3823,45362,3681,45193,0.084278,-3.354299,1.542883,0.122859,FALSE
4,2,add_payment_info / session,2344,50637,2409,50244,0.046290,3.576911,-1.240994,0.214608,FALSE
5,2,add_shipping_info / session,3480,50637,3510,50244,0.068724,1.650995,-0.709557,0.477979,FALSE
6,2,begin_checkout / session,4262,50637,4313,50244,0.084168,1.988164,-0.952898,0.340642,FALSE
7,2,new_accounts / session,4165,50637,4184,50244,0.082252,1.241934,-0.588793,0.556000,FALSE
8,3,add_payment_info / session,3623,70047,3697,70439,0.051722,1.474630,-0.643172,0.520112,FALSE
9,3,add_shipping_info / session,5298,70047,5188,70439,0.075635,-2.621211,1.413727,0.157442,FALSE


In [ ]:
test_ab_.to_csv("/content/drive/MyDrive/Mate_homework/test_ab.csv", index=False)

 [Dashboard](https://public.tableau.com/app/profile/aniuta.muliarchuk/viz/ABTestingTool_17397096719320/ABTestingTool?publish=yes)

 [ csv файл](https://drive.google.com/file/d/1aQxDgXvmzIlukN4vSr-n8RhaEefe80Ye/view?usp=share_link)